In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout
import tensorflow as tf
import random
from sklearn.model_selection import TimeSeriesSplit
import statsmodels.api as sm

import xgboost as xgb
from sklearn.model_selection import train_test_split

2024-09-09 06:55:48.437087: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-09 06:55:48.437234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-09 06:55:48.607166: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv('../input/amirkabirdam/dam12.csv')
df.head(2)

,DamId,DateRow,Torbin_out,TakhlieAbgir_out,Sarriz_out,TabKhir_out,Total_out,TotalMasraf_out,TakhlieRosob_out,Cheshme_out,...,NamNesbi6_5,NamNesbi12_5,NamNesbi18_5,TolGeo,ArzGeo,MaxErtefaAzDarya,MinErtefaAzDarya,KafDaryache,GonjayeshKolMakhzan,GonjayeshMofidMakhzan
0,1,1966-05-11,1.63296,0.0,0.0,0.0,1.63296,0.22464,0,0,...,0.0,0.0,0.0,54.25563,35.252621,1765,1600,1595,183,177.3
1,1,1970-10-08,0.78624,0.0,0.0,0.0,0.78624,0.45792,0,0,...,0.0,0.0,0.0,54.25563,35.252621,1765,1600,1595,183,177.3


In [3]:
df['humidity']=df[['NamNesbi6_5', 'NamNesbi12_5','NamNesbi18_5']].mean(axis=1)
df['daraje']= df[['MinDarajehararat_in','MaxDarajehararat_in']].mean(axis=1)

In [4]:
df['DateRow'] = pd.to_datetime(df['DateRow'])
df = df.sort_values(by= 'DateRow')
df['year'] = df['DateRow'].dt.year
df['month'] = df['DateRow'].dt.month
df['day_of_week'] = df['DateRow'].dt.dayofweek
df['day_of_year'] = df['DateRow'].dt.dayofyear

In [5]:
df['Total_in1']=df['Total_in'].shift(1)
df['Total_in7']= df['Total_in'].shift(7)
df['Total_in30']= df['Total_in'].shift(30)

df['Barandegi_in1']=df['Barandegi_in'].shift(1)
df['Barandegi_in7']=df['Barandegi_in'].shift(7)
df['Barandegi_in30']=df['Barandegi_in'].shift(30)
df['daraje1']=df['daraje'].shift(1)
df['daraje7']=df['daraje'].shift(7)
df['daraje30']=df['daraje'].shift(30)

In [6]:
df['rolling_mean_7'] = df['Total_in'].shift(1).rolling(window=7).mean()
df['rolling_std_7'] = df['Total_in'].shift(1).rolling(window=7).std()
df['rolling_mean_30'] = df['Total_in'].shift(7).rolling(window=30).mean()
df['rolling_std_30'] = df['Total_in'].shift(7).rolling(window=30).std()

In [7]:
df.dropna(axis=0, inplace=True)

In [8]:
target = 'Total_in'
features = ['Total_in1','Total_in7','Total_in30','Barandegi_in','Barandegi_in1','Barandegi_in7'
            ,'Barandegi_in30','TotalAbBarfVBaran_in','DebiSeilabm3c','daraje','daraje1','daraje7'
            ,'daraje30','VazeHava_in','SoratBad_in' ,'humidity'
           ,'rolling_mean_7','rolling_std_7','rolling_mean_30','rolling_std_30']

In [9]:
df = df[df['DateRow']>'2015']
df.drop(columns=['DateRow'], inplace=True)

y=df[target]

x = df[features]

In [10]:
print("x:", x.shape ,"y:",y.shape)

x: (2936, 20) y: (2936,)


In [11]:
tscv = TimeSeriesSplit(n_splits = 7)
train_rmse = []
test_rmse = []
train_r2 = []
test_r2 = []
i=0
for train_index, test_index in tscv.split(x):
    i+=1
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    if i>=3:
        model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.01)
        model.fit(X_train, Y_train, verbose=False)

    # Predictions
        preds = model.predict(X_test)
       
        test_rmse.append ( np.sqrt(mean_squared_error(Y_test, preds)))
        
        test_r2 .append ( r2_score(Y_test, preds))
        
print("rmse:",np.mean(test_rmse) , "r2",np.mean(test_r2) )

rmse: 0.4650533015860804 r2 0.7587137891861258
